In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [2]:
fake=pd.read_csv(r'C:\Users\Zankhna\Desktop\Fake News Detection\Fake (1).csv')
true=pd.read_csv(r'C:\Users\Zankhna\Desktop\Fake News Detection\True.csv')

In [3]:
from PIL import Image
import numpy as np

def preprocess_image(img_file):
    image = Image.open(img_file).convert('RGB')
    image = image.resize((224, 224))  # Adjust based on your model
    image_array = np.array(image) / 255.0
    return image_array.reshape(1, 224, 224, 3)  # Add batch dimension


In [4]:
pip install tensorflow

In [6]:
import tensorflow as tf



In [7]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
fake['class']=0
true['class']=1

In [10]:
data=pd.concat([fake,true],axis=0)

In [11]:
data.sample(10)


,title,text,subject,date,class
21449,EXPOSE THE LIES: Shut Down Planned Parenthood’...,TODAY IS SCHEDULE YOUR MAMMOGRAM DAY WITH PL...,left-news,"Sep 21, 2015",0
21162,YOU BE THE JUDGE: WATCH…Did Obama Put Somethin...,Are you kidding me? When is the last time you ...,left-news,"Jan 5, 2016",0
8778,Trump seeks $10 million from former adviser in...,(Reuters) - Republican White House hopeful Don...,politicsNews,"July 13, 2016",1
3561,Breitbart’s War On Kellog Backfires SPECTACUL...,When Kellogg pulled its advertising from the w...,News,"December 4, 2016",0
19720,Syrian army sends more troops across Euphrates...,BEIRUT (Reuters) - Syrian troops crossed to th...,worldnews,"September 18, 2017",1
14615,BREAKING: NORTH KOREA Detains American Student...,Apparently it takes just one American student ...,politics,"Jan 22, 2016",0
564,New judge assigned to U.S. lawsuit against AT&...,WASHINGTON (Reuters) - The U.S. Department of ...,politicsNews,"November 21, 2017",1
1920,Trump Embarrasses Himself With Yet Another De...,On Sunday morning. Donald Trump once again rev...,News,"April 2, 2017",0
20287,Italy court deals blow to 5-Star ahead of Sici...,ROME (Reuters) - A court in Sicily has suspend...,worldnews,"September 12, 2017",1
2070,Trump’s Terrible Monday Has A Major Historian...,Donald Trump must be forced to resign or be im...,News,"March 21, 2017",0


In [12]:
data=data.drop(['title','subject','date'],axis = 1)

In [13]:
data.reset_index(inplace=True)

In [14]:
data.drop(['index'],axis=1,inplace=True)

In [15]:
data.sample(5)

,text,class
41367,YANGON (Reuters) - Myanmar on Tuesday launched...,1
41380,HANOI (Reuters) - Vietnam called on Cambodia o...,1
39508,ABUJA (Reuters) - Nigeria s cabinet will be ex...,1
19315,IT S TIME TO DRAIN OBAMA S SWAMP! Rep. Jason C...,0
28665,WASHINGTON (Reuters) - Senior Cabinet members ...,1


In [16]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[."?\]',"",text)
    text = re.sub("\\W"," ",text)
    text = re.sub("https?:://\S+|www\.\S+","",text)
    text = re.sub("<.*?> +","", text)
    text = re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text = re.sub("\n","",text)
    text = re.sub("\w*\d\w*","",text)
    return text
 

In [17]:
data['text'] = data['text'].apply(clean_text)

In [18]:
x=data['text']
y=data['class']

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.25,random_state=42)

In [19]:
vectorizer =TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [20]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [21]:
prediction= lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9861024498886414

In [22]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [23]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']